## EfficientNet Architecture

In [ ]:
import torch
import torch.nn as nn

import torchvision

In [ ]:
class CNNBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, groups=1,act=True, bias=False):

        super().__init__()
        # same padding quick mapping:
        # k=1 -> p =0, k=3 -> p=1, k=5 -> p=2
        padding = kernel_size // 2
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding = padding, bias=bias, groups=groups)
        self.batch_norm = nn.BatchNorm2d(out_channels)
        self.activation = nn.SiLU() if act else nn.Identity()

    def forward(self, x):
        x= self.conv(x)
        x= self.batch_norm(x)
        return self.activation(x)

In [ ]:
class SqueezeAndExcitationBlock(nn.Module):
    def __init__(self, in_channels,reduction_ratio=16): #reduction_ratio to reduce computation, a hyperparameter. take r=16 for balance in complexity and capacity as in SeNet paper
        super().__init__()
        self.squeeze = nn.AdaptiveAvgPool2d((1,1))
        self.fc1 = nn.Linear(in_channels , in_channels//reduction_ratio, bias=False)
        self.relu = nn.ReLU(inplace=True)
        self.fc2 = nn.Linear(in_channels//reduction_ratio, in_channels, bias =False)
        self.sigmoid = nn.Sigmoid()


    def forward(self,x):

        x_out = self.squeeze(x)
        x_out = torch.flatten(x_out,1)
        x_out = self.relu(self.fc1(x_out))
        x_out = self.sigmoid(self.fc2(x_out))

        x_out = x_out[:,:,None,None]

        scaled = x * x_out
        return scaled


In [ ]:
# class SqueezeAndExcitationBlockUncheckedLikelyWrong(nn.Module):
#     def __init__(self, in_channels, reduction_ratio=16):
#         super().__init__()
#         self.squeeze = nn.AdaptiveAvgPool2d(1)
#         self.excitation = nn.Sequential(
#             nn.Linear(in_channels, in_channels // reduction_ratio, bias=False),
#             nn.ReLU(inplace=True),
#             nn.Linear(in_channels // reduction_ratio, in_channels, bias=False),
#             nn.Sigmoid()
#         )

#     def forward(self, x):
#         batch, channels, _, _ = x.size()
#         y = self.squeeze(x).view(batch, channels)
#         y = self.excitation(y).view(batch, channels, 1, 1)
#         return x * y


In [ ]:
class MobileConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, expansion_ratio, reduction_ratio):
        super().__init__()

        exp_out_channels = in_channels * expansion_ratio

        #residual connection only when it is not being downsampled in any way
        self.add_res = in_channels == out_channels and stride ==1
        self.conv1 = CNNBlock(in_channels, exp_out_channels, 1,1) if expansion_ratio > 1 else nn.Identity()

        #depthwise convolution
        self.conv2 = CNNBlock(exp_out_channels, exp_out_channels, kernel_size, stride, exp_out_channels)
        self.se = SqueezeAndExcitationBlock(exp_out_channels,reduction_ratio)
        self.conv3 = CNNBlock(exp_out_channels,out_channels, 1,1, act=False) #hatched line features means no activation

        self.sd = StochasticDepth(0.75)

    def forward(self, x):
        x_out = self.conv3(self.se(self.conv2(self.conv1(x))))

        if self.add_res:
            x_out = x + x_out

        x_out = self.sd(x_out)

        return x_out

In [ ]:
class ClassificationBlock(nn.Module):
    def __init__(self,in_channels, n_classes, dropout_prob):
        super().__init__()
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.dropout = nn.Dropout(dropout_prob)
        self.fc = nn.Linear(in_channels, n_classes)
        self.sigmoid = nn.Sigmoid()

    def forward(self,x):
        x = self.avgpool(x)
        x = self.dropout(x)
        x= torch.flatten(x,1)
        x = self.fc(x)
        return self.sigmoid(x)


In [ ]:
class StochasticDepth(nn.Module):
    def __init__(self, p=0.75):
        super().__init__()
        self.p = p

    def forward(self, x):

        rand_mask = torch.rand((x.shape[0], 1,1,1),  dtype=x.dtype, device=x.device)
        binary_mask = torch.floor(rand_mask) #TODO

        if self.training:   x = x/self.p * binary_mask

        return x

In [ ]:
class EffNet(nn.Module):
    def __init__(self, model_name, in_channels=3, n_classes=25, show_output_dims=False):
        super().__init__()
        self.show = show_output_dims
        self.model_name = model_name
        self.config = Config()
        self.stages = self.config.stages
        self.phis = self.config.phis[model_name]

        #parameters
        phi, res, dropout_p = self.phis
        self.calc_coeffs(phi)

        #define network
        self.network = nn.ModuleList([])
        self.channels =[]

        #baseline stage 1
        operator, channels, layers, kernel_size, stride, expansion_ratio = self.config.stages[0]
        self.add_layers(3, operator, channels, layers, kernel_size, stride) #rgb input layer, TODO: check for errors
        print(operator)

        #remaining stages: 9 stages ko 7 stages (2-8) lai
        for i in range(1, len(self.stages)-1):

            if i==1:
                reduction_ratio=4
            else:
                reduction_ratio=24

            operator, channels, layers, kernel_size, stride, expansion_ratio = self.config.stages[i]
            self.add_layers(self.channels[-1], operator, channels, layers, kernel_size, stride, expansion_ratio, reduction_ratio)
            print(operator)


        #final stage: conv1x1 and classifier
        operator, channels, layers, kernel_size, stride, expansion_ratio = self.config.stages[-1]
        self.add_layers(self.channels[-1], operator, channels, layers, kernel_size, stride) #the conv layer
        print(operator)
        self.network.append(ClassificationBlock(self.channels[-1], n_classes,dropout_p)) #the classifier block



    def forward(self, x):

        for stage_num,module in enumerate(self.network):

            x= module(x)

            shape = x.shape
            if self.show: print(f"shape of stage{stage_num} : {shape}")

        return x


    def add_layers(self, in_channels, operator, channels, layers, kernel_size, stride, *args):

        channels, layers = self.update_dw(channels, layers)

        if layers == 1:
            self.network.append(operator(in_channels, channels, kernel_size, stride, *args))
        else:
            #the first
            self.network.append(operator(in_channels, channels, kernel_size, 1, *args))

            #the remaining except first and last: works if there are >3 layers
            for _ in range(layers-2):
                self.network.append(operator(channels, channels,kernel_size, 1, *args))

            #final layer with stride dependent on the stage
            self.network.append(operator(channels,channels, kernel_size, stride, *args))

        self.channels.append(channels)

    # for models higher than the basseline:

    def calc_coeffs(self, phi, alpha=1.2, beta =1.1): #alpha and beta from EffNet paper, calculated through grid search. We dont use gamma but use the resolution from config
        # in every higher model, the channels is multiplied by width (beta^phi) layers is multiplied by depth (alpha^phi)
        self.depth = alpha ** phi
        self.width = beta ** phi

    def update_dw(self,channels, layers):
        return int(channels * self.width), int(layers * self.depth)



In [ ]:
class Config:
    stages = [
            # [Operator(F), Channels, Layers, Kernel, Stride, Expansion Ratio]
            [CNNBlock, 32, 1, 3, 2, 1],
            [MobileConvBlock, 16, 1, 3, 1, 1],
            [MobileConvBlock, 24, 2, 3, 2, 6],
            [MobileConvBlock, 40, 2, 5, 2, 6],
            [MobileConvBlock, 80, 3, 3, 2, 6],
            [MobileConvBlock, 112, 3, 5, 1, 6],
            [MobileConvBlock, 192, 4, 5, 2, 6],
            [MobileConvBlock, 320, 1, 3, 1, 6],
            [CNNBlock, 1280, 1, 1, 1, 0]
    ]

    phis = {
            # BX : (phi, resolution, dropout)
            "B0" : (0, 224, 0.2),
            "B1" : (0.5, 240, 0.2),
            "B2" : (1, 260, 0.3),
            "B3" : (2, 300, 0.3),
            "B4" : (3, 380, 0.4),
            "B5" : (4, 456, 0.4),
            "B6" : (5, 528, 0.5),
            "B7" : (6, 600, 0.5)
    }

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model_name ="B0"
model = EffNet(model_name = model_name, in_channels = 3, n_classes = 25, show_output_dims=False).to(device)
res = Config().phis[model_name][1]
print(res)

x= torch.randn((5, 3, res, res)).to(device)
out= model(x)
print(out.shape)

In [ ]:
!pip install torch-summary

In [ ]:
from torchsummary import summary

In [ ]:
_ =summary(model,x)

## 2. Training, Validating and Saving the model

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import transforms, ToTensor
from tqdm import tqdm

In [ ]:
train_dir = "/kaggle/input/seen-dataset-12-class/Seen Datasets/train"
val_dir = "/kaggle/input/seen-dataset-12-class/Seen Datasets/val"
test_dir =""

In [ ]:
#data loaders for mean calculation
transforms_func = transforms.Compose([transforms.Resize((224,224)),
                                      ToTensor()])
train_ds= datasets.ImageFolder(train_dir, transform=transforms_func)
batch_size = 32
train_dataloader = DataLoader(train_ds, batch_size=batch_size,shuffle=False)


In [ ]:
def get_ds_mean_std(dataloader):
    mean= 0.0 # torch.zeros(3)
    var= 0.0 # torch.zeros(3)
    n_imgs_total=0
    i=0
    for images, _ in dataloader:

        n_imgs_per_batch = images.shape[0]
#         print(images)
        #b,c,w*h
        channelwise_images = images.view(n_imgs_per_batch, images.shape[1], -1)
        #mean and var per channel, summed every batch
        mean += channelwise_images.mean(2).sum(0)

        var += channelwise_images.var(2).sum(0)

        n_imgs_total += n_imgs_per_batch

    #means can simply be averaged
    mean /=n_imgs_total

    #std can't be averaged. but, for equal batch sizes (only one batch might have different size)
    std = torch.sqrt(var/n_imgs_total)
    print(n_imgs_total)
    return mean,std


In [ ]:
# get_ds_mean_std(train_dataloader)

In [ ]:
#output of above
ds_mean, ds_std = [0.4731, 0.4819, 0.4018], [0.3544, 0.3544, 0.3544]

In [ ]:
transforms_func = transforms.Compose([transforms.Resize((224,224)),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.RandomRotation(10),
                                      ToTensor(),
                                      transforms.Normalize(mean=ds_mean, std=ds_std)])

train_ds= datasets.ImageFolder(train_dir, transform=transforms_func)
# test_ds= datasets.ImageFolder(test_dir, transform=transforms_func)
val_ds= datasets.ImageFolder(val_dir, transform=transforms_func)

#data loaders for training

batch_size = 25

train_dataloader = DataLoader(train_ds, batch_size=batch_size,shuffle=True)
# test_dataloader = DataLoader(test_ds, batch_size=batch_size,shuffle=False)
val_dataloader = DataLoader(val_ds, batch_size=batch_size,shuffle=True)

In [ ]:
# for labels,image in train_dataloader:
#     print(image)

### Training Loop

In [ ]:
n_epochs= 10
#scheduler ni chaine ho? for lr decay. edit: Hola, lets see. TODO: copy paper's

optimizer = torch.optim.RMSprop(model.parameters(), lr=0.256, alpha=0.9, momentum=0.9, weight_decay = 1e-5)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=2, eta_min=1e-5)
criterion = nn.CrossEntropyLoss()

In [ ]:
a=optimizer.param_groups
a[0].keys()


In [ ]:
#required
n_training_samples =22500

In [ ]:
a=torch.randn(5,25)
a

In [ ]:
a.argmax(dim=1, keepdim=True)

In [ ]:
def get_lr(optimizer):
    #list of param groups
    return optimizer.param_groups[0]["lr"]

# def get_lr(optimizer):
#     for param_group in optimizer.param_groups:
#         return param_group["lr"]

print(get_lr(optimizer))

In [ ]:
#get the number of correct predictions per batch
(5,25)
def count_correct_batch(output, target):
    pred = output.argmax(dim=1, keepdim=True)
    print(f"highest pred class: {pred}")
    n_correct =pred.eq(target.view_as(pred)).sum().item()
    return n_correct

In [ ]:
#calculate loss values per batch of data

def loss_batch(criterion, result, target, optimizer=None):

    #get loss
    loss=criterion(result,target)
    print(f"batch loss:{loss} ")
    #get performance metric
    print(f"result: {result}")
    print(f"target: {target}")
    n_correct_b = count_correct_batch(result,target)

    print(f"n_correct per batch: {n_correct_b}")

    if optimizer is not None:
        optimizer.zerograd()
        loss.backward()
        opt.step()

    return loss.item(), n_correct_b

In [ ]:
from tqdm import tqdm

In [ ]:
def loss_acc_epoch(model, criterion, dataloader, len_data, check_id=False, optimizer=None):

    running_loss =0.0
    running_n_correct =0.0
    len_data = len(dataloader)

    for images, labels in tqdm(dataloader):

        images = images.to(device)
        labels = labels.to(device)

        result = model(images)

        loss_b, n_correct_b = loss_batch(criterion,result,labels, optimizer)

        running_loss += loss_b

        if(n_correct_b is not None):
            running_n_correct += n_correct_b

        if(check_id):
            break #stop when checking


    loss_epoch = running_loss/float(len_data)
    acc_epoch = running_n_correct/float(len_data)

    return loss_epoch, acc_epoch


In [ ]:
import copy

In [ ]:
def train(model,train_dataloader, val_dataloader, criterion, optimizer, n_epoch, device, scheduler,check_id, save_path, val_iter = 5):

    loss_hist = {
        "train": [],
        "val": []
    }

    acc_hist={
        "train": [],
        "val": []
    }


    #copy the current model wts as best model:
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss=float('inf')
    step=0
    #TODO:

    for epoch in range(n_epochs):

        print(f"Epoch {epoch+1} of {n_epochs}")
        current_lr = get_lr(optimizer)
        step=0


        model.train()
        train_loss_epoch, train_acc_epoch = loss_acc_epoch(model, criterion, train_dataloader, check_id, optimizer)

        loss_hist["train"].append(train_loss_epoch)
        acc_hist["train"].append(train_acc_epoch)
        step+=1

        #evaluation now

        model.eval()
        with torch.no_grad():
            val_loss_epoch, val_acc_epoch = loss_acc_epoch(model,criterion, val_dataloader, check_id)
            loss_hist["val"].append(val_loss_epoch)
            acc_hist["val"].append(val_acc_epoch)

        #every epoch, keep on selecting the best model sofar
        if val_loss_epoch < best_loss:
            best_loss = val_loss_epoch
            best_model_wts = copy.deepcopy(model.state_dict())

            torch.save(model.state_dict(), "savemodel.pt")
            print("best model saved")

        scheduler.step()
        print(f"Tr.Loss: {train_loss_epoch:.5f}, Tr.Acc:{train_acc_epoch:.5f} ... Val.loss: {val_loss_epoch}, Val.Acc: {val_acc_epoch:.5f} ")


    return loss_hist, acc_hist

In [ ]:
save_path = "/kaggle/working/"

In [ ]:
from torchvision import models, transforms

In [ ]:
model = models.efficientnet_b5(pretrained=False)

# Modify the classifier to have 6 output classes instead of 1000
model.classifier[1] = nn.Linear(model.classifier[1].in_features, 25)

def initialize_weights(m):
    if isinstance(m, nn.Conv2d):
        nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
        if m.bias is not None:
            nn.init.constant_(m.bias, 0)
    elif isinstance(m, nn.BatchNorm2d):
        nn.init.constant_(m.weight, 1)
        nn.init.constant_(m.bias, 0)
    elif isinstance(m, nn.Linear):
        nn.init.normal_(m.weight, 0, 0.01)
        nn.init.constant_(m.bias, 0)

# Apply the weight initialization
model.apply(initialize_weights)

In [ ]:
model.to(device)

In [ ]:
device

In [ ]:
# loss_hist, acc_hist = train(model,train_dataloader, val_dataloader, criterion, optimizer, n_epochs=10, device=device, scheduler=scheduler,check_id=False, save_path=save_path,)

In [ ]:
save_path='model.pt'

In [ ]:
def train(model, criteria, optimizer, n_epochs, train_dataloader, val_dataloader, device, val_iter=10):

    train_losses=[]
    train_accs=[]
    val_losses=[]
    val_accs=[]

    for epoch in range(1, n_epochs+1):

        print(f"Epoch {epoch} of {n_epochs}:")

        #------training------------

        n_train_samples = len(train_dataloader.dataset)

        #train mode activate
        model.train()
        epoch_train_loss = 0
        correct_preds=0
        epoch_train_acc = float(0)
        step=0


        #all the training that happens per epoch
        for images, targets in tqdm(train_dataloader):
            #iterates through the dataset in batches

            images = images.to(device)
            targets = targets.to(device)
            optimizer.zero_grad()

            #forward pass
            results = model(images)
            losses = criteria(results, targets.unsqueeze(1).float())

            #backward pass
            losses.backward()
            optimizer.step()

            epoch_train_loss += losses.item()
            epoch_train_acc += torch.sum(results.round() == targets.unsqueeze(1).float())

            step +=1

        #after going through the entire dataset calculate the train loss and accuracy
        epoch_train_loss /= step
        train_losses.append(epoch_train_loss)

        epoch_train_acc /= float(n_train_samples)
        train_accs.append(epoch_train_acc)

        print(f"Epoch {epoch} Training Loss: {epoch_train_loss:.4f}")
        print(f"Epoch {epoch} Training Acc: {epoch_train_acc:.4f}")


        #perform validation every few epochs
        if epoch % val_iter ==0:

            #------validation------------
            n_val_samples = len(val_dataloader.dataset)

            #evaluation mode activate
            model.eval()
            epoch_val_loss =0
            epoch_val_acc = float(0)
            val_step=0

            with torch.no_grad():
                for images, targets in tqdm(val_dataloader):
                    #iterates through the dataset in batches

                    images = images.to(device)
                    targets = targets.to(device)
                    optimizer.zero_grad()

                    #forward pass
                    results = model(images)
                    losses = criteria(results, targets.unsqueeze(1).float())

                    #no backward pass

                    epoch_val_loss += losses.item()

                    #rounding basically thresholds the output at 0.5
                    epoch_val_acc += torch.sum(results.round() == targets.unsqueeze(1).float())

                    val_step +=1

                #after going through the entire dataset calculate the validation loss and accuracy
                epoch_val_loss /= val_step
                val_losses.append(epoch_val_loss)
                epoch_val_acc /= n_val_samples
                val_accs.append(epoch_val_acc)

        print(f"Epoch {epoch} Validation Loss: {epoch_train_loss:.4f}")
        print(f"Epoch {epoch} Validation Acc: {epoch_train_acc:.4f}")

    return train_losses, val_losses, train_accs, val_accs

In [ ]:
import copy
import torch
from tqdm import tqdm

def loss_acc_epoch(model, criterion, dataloader, check_id, device, optimizer=None):
    epoch_loss = 0
    correct_preds = 0
    total_samples = len(dataloader.dataset)
    steps = 0

    for images, targets in tqdm(dataloader):
        images = images.to(device)
        targets = targets.to(device)

        if optimizer:
            optimizer.zero_grad()

        results = model(images)
        losses = criterion(results, targets)

        if optimizer:
            losses.backward()
            optimizer.step()

        epoch_loss += losses.item()
        correct_preds += (results.argmax(dim=1) == targets).sum().item()
        steps += 1

    epoch_loss /= steps
    epoch_acc = correct_preds / total_samples

    return epoch_loss, epoch_acc

def train(model, train_dataloader, val_dataloader, criterion, optimizer, n_epochs, device, scheduler, check_id, save_path, val_iter=5):
    loss_hist = {"train": [], "val": []}
    acc_hist = {"train": [], "val": []}

    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = float('inf')

    for epoch in range(1, n_epochs + 1):
        print(f"Epoch {epoch} of {n_epochs}")

        model.train()
        train_loss_epoch, train_acc_epoch = loss_acc_epoch(model, criterion, train_dataloader, check_id, device, optimizer)

        loss_hist["train"].append(train_loss_epoch)
        acc_hist["train"].append(train_acc_epoch)

        if epoch % val_iter == 0 or epoch == n_epochs:
            model.eval()
            with torch.no_grad():
                val_loss_epoch, val_acc_epoch = loss_acc_epoch(model, criterion, val_dataloader, check_id, device)
                loss_hist["val"].append(val_loss_epoch)
                acc_hist["val"].append(val_acc_epoch)

            if val_loss_epoch < best_loss:
                best_loss = val_loss_epoch
                best_model_wts = copy.deepcopy(model.state_dict())
                torch.save(model.state_dict(), save_path)
                print("Best model saved")

            print(f"Epoch {epoch} Validation Loss: {val_loss_epoch:.4f}, Validation Acc: {val_acc_epoch:.4f}")

        scheduler.step()
        print(f"Training Loss: {train_loss_epoch:.4f}, Training Acc: {train_acc_epoch:.4f}")

    model.load_state_dict(best_model_wts)
    return loss_hist, acc_hist

# Example usage (make sure to replace variables with actual values):
# model, criterion, optimizer, n_epochs, device, scheduler, check_id, save_path = ...
# loss_hist, acc_hist = train(model, train_dataloader, val_dataloader, criterion, optimizer, n_epochs, device, scheduler, check_id, save_path)
